In [1]:
# importing pandas and numpy
import pandas as pd
import numpy as np

In [2]:
# importing pyspark
import pyspark

In [3]:
# importing the spark session
from pyspark.sql import SparkSession

In [4]:
# creating the seasion
spark = SparkSession.builder.appName("Walmart Ecommerce Product Details").getOrCreate()

22/10/20 10:32:04 WARN Utils: Your hostname, HP-G62 resolves to a loopback address: 127.0.1.1; using 192.168.18.113 instead (on interface enp3s0)
22/10/20 10:32:04 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/20 10:32:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
# Read JSON file into dataframe
walmart_df = spark.read.json("walmart_ecommerce_product_details.json")

In [6]:
walmart_df.show()

+---------+--------------------+--------------------+--------------------+--------------------+------------+-----------+----------+---------------+-----------+--------------------+--------------------+----------+--------------------+---------------+
|Available|               Brand|            Category|     Crawl Timestamp|         Description|        Gtin|Item Number|List Price|   Package Size|Postal Code|        Product Name|         Product Url|Sale Price|             Uniq Id|_corrupt_record|
+---------+--------------------+--------------------+--------------------+--------------------+------------+-----------+----------+---------------+-----------+--------------------+--------------------+----------+--------------------+---------------+
|     TRUE|       In Style Eyes|Health|Home Healt...|2019-03-22 17:10:...|Stunning Looking ...|096647824338|           |     19.99|               |           |In Style Eyes Cat...|https://www.walma...|     19.99|459b05f3cb7f1cba0...|           null|


In [22]:
 # making different product dataframe from walmart_df
product_df= walmart_df.select(["Brand","Product Name","Description","List Price","Sale Price","Category","Available"])

In [23]:
import pyspark.sql.functions as F 
from pyspark.sql import Window
import pyspark.sql.types as T

In [24]:
# 1. Get the Brand along with products associated with it.

product_df.na.drop("all")
product_df.show()
# but

+--------------------+--------------------+--------------------+----------+----------+--------------------+---------+
|               Brand|        Product Name|         Description|List Price|Sale Price|            Category|Available|
+--------------------+--------------------+--------------------+----------+----------+--------------------+---------+
|       In Style Eyes|In Style Eyes Cat...|Stunning Looking ...|     19.99|     19.99|Health|Home Healt...|     TRUE|
|       In Style Eyes|In Style Eyes Cat...|Stunning Looking ...|     19.99|     19.99|Health|Home Healt...|     TRUE|
|       In Style Eyes|In Style Eyes Cat...|Stunning Looking ...|     19.99|     19.99|Health|Home Healt...|     TRUE|
|       In Style Eyes|In Style Eyes Cat...|Stunning Looking ...|     19.99|     19.99|Health|Home Healt...|     TRUE|
|         Upper Crust|Upper Crust Carib...|| Size informatio...|     39.96|     39.96|Food|Baking|Bakin...|    FALSE|
|     Unique Bargains|Fish Tank Ornamen...| Material: Pl

In [25]:
#  making windows spec for partition by Brand 
windowSpec=Window.partitionBy("Brand")

In [26]:
win_brand_df=product_df.withColumn("Product List",F.collect_list(F.col("Product Name")).over(windowSpec))
win_brand_df.show()

+-------------------+--------------------+--------------------+----------+----------+--------------------+---------+--------------------+
|              Brand|        Product Name|         Description|List Price|Sale Price|            Category|Available|        Product List|
+-------------------+--------------------+--------------------+----------+----------+--------------------+---------+--------------------+
|               null|                null|                null|      null|      null|                null|     null|                  []|
|          100% Pure|100% Pure Long La...|100% Pure Long La...|        30|        30|Beauty|Makeup|Eye...|    FALSE|[100% Pure Long L...|
|          32 Square|32 Square DIL14GW...||Your little pale...|    143.03|    143.03|Toys|Games & Puzz...|    FALSE|[32 Square DIL14G...|
|                 3M|1/4" x 24" Coated...|Sanding Belt, Wid...|    226.99|    226.99|Home Improvement|...|     TRUE|[1/4" x 24" Coate...|
|             3dRose|3dRose Sarcas

In [27]:
# 2. List all the product names whose list price is greater than sales price

product_df.na.drop("all")
product_df.show()
# but


+--------------------+--------------------+--------------------+----------+----------+--------------------+---------+
|               Brand|        Product Name|         Description|List Price|Sale Price|            Category|Available|
+--------------------+--------------------+--------------------+----------+----------+--------------------+---------+
|       In Style Eyes|In Style Eyes Cat...|Stunning Looking ...|     19.99|     19.99|Health|Home Healt...|     TRUE|
|       In Style Eyes|In Style Eyes Cat...|Stunning Looking ...|     19.99|     19.99|Health|Home Healt...|     TRUE|
|       In Style Eyes|In Style Eyes Cat...|Stunning Looking ...|     19.99|     19.99|Health|Home Healt...|     TRUE|
|       In Style Eyes|In Style Eyes Cat...|Stunning Looking ...|     19.99|     19.99|Health|Home Healt...|     TRUE|
|         Upper Crust|Upper Crust Carib...|| Size informatio...|     39.96|     39.96|Food|Baking|Bakin...|    FALSE|
|     Unique Bargains|Fish Tank Ornamen...| Material: Pl

In [28]:
win_price_df=product_df.withColumn("Product List",F.collect_list(F.col("Product Name")).over(windowSpec))
win_price_df.filter(F.col("List Price") > F.col("Sale Price")).show()

+-------------------+--------------------+--------------------+----------+----------+------------------+---------+--------------------+
|              Brand|        Product Name|         Description|List Price|Sale Price|          Category|Available|        Product List|
+-------------------+--------------------+--------------------+----------+----------+------------------+---------+--------------------+
|4Legs4Pets by Mahar|4Legs4Pets by Mah...|Features: -Frame ...|     51.87|     38.32|Pets|Dogs|Dog Beds|    FALSE|[4Legs4Pets by Ma...|
|4Legs4Pets by Mahar|4Legs4Pets by Mah...|Features: -Frame ...|     64.85|     47.94|Pets|Dogs|Dog Beds|    FALSE|[4Legs4Pets by Ma...|
|4Legs4Pets by Mahar|4Legs4Pets by Mah...|Features: -Frame ...|     46.66|     34.46|Pets|Dogs|Dog Beds|    FALSE|[4Legs4Pets by Ma...|
|4Legs4Pets by Mahar|4Legs4Pets by Mah...|Features: -Frame ...|     43.73|     32.29|Pets|Dogs|Dog Beds|    FALSE|[4Legs4Pets by Ma...|
|4Legs4Pets by Mahar|4Legs4Pets by Mah...|Featur

In [29]:
# 3. Count the number of product names whose list price is greater than sales price
win_count_df=product_df.withColumn("no. of Product",F.count(F.col("Product Name")).over(windowSpec))
win_count_df.filter(F.col("List Price") > F.col("Sale Price")).show()

+-------------------+--------------------+--------------------+----------+----------+------------------+---------+--------------+
|              Brand|        Product Name|         Description|List Price|Sale Price|          Category|Available|no. of Product|
+-------------------+--------------------+--------------------+----------+----------+------------------+---------+--------------+
|4Legs4Pets by Mahar|4Legs4Pets by Mah...|Features: -Frame ...|     51.87|     38.32|Pets|Dogs|Dog Beds|    FALSE|            85|
|4Legs4Pets by Mahar|4Legs4Pets by Mah...|Features: -Frame ...|     64.85|     47.94|Pets|Dogs|Dog Beds|    FALSE|            85|
|4Legs4Pets by Mahar|4Legs4Pets by Mah...|Features: -Frame ...|     46.66|     34.46|Pets|Dogs|Dog Beds|    FALSE|            85|
|4Legs4Pets by Mahar|4Legs4Pets by Mah...|Features: -Frame ...|     43.73|     32.29|Pets|Dogs|Dog Beds|    FALSE|            85|
|4Legs4Pets by Mahar|4Legs4Pets by Mah...|Features: -Frame ...|     59.38|      47.5|Pets|

In [17]:
# 4. List all the products belong to a “women” category.
#  making windows spec for partition by Category 
windowSpec=Window.partitionBy("Category")
win_women_df=product_df.withColumn("Product List",F.collect_list(F.col("Product Name")).over(windowSpec))\
    .filter(F.col("Category")=="Women")
win_women_df.show()

+-----+------------+----------+----------+--------+---------+------------+
|Brand|Product Name|List Price|Sale Price|Category|Available|Product List|
+-----+------------+----------+----------+--------+---------+------------+
+-----+------------+----------+----------+--------+---------+------------+



In [33]:
# 5. List the products which are available.
#  making windows spec for partition by Brand 
windowSpec=Window.partitionBy("Brand")
win_available_df=product_df.withColumn("Product List",F.collect_list(F.col("Product Name")).over(windowSpec))\
    .filter(F.col("Available")=="TRUE")
win_available_df.show()

+-------------------+--------------------+--------------------+----------+----------+--------------------+---------+--------------------+
|              Brand|        Product Name|         Description|List Price|Sale Price|            Category|Available|        Product List|
+-------------------+--------------------+--------------------+----------+----------+--------------------+---------+--------------------+
|                 3M|1/4" x 24" Coated...|Sanding Belt, Wid...|    226.99|    226.99|Home Improvement|...|     TRUE|[1/4" x 24" Coate...|
|             3dRose|3dRose Pink Teddy...|Pink Teddy Bear- ...|     16.18|     16.18|Party & Occasions...|     TRUE|[3dRose Sarcasm n...|
|             3dRose|3dRose USA, Alask...|USA, Alaska. Orca...|     10.22|     10.22|Arts, Crafts & Se...|     TRUE|[3dRose Sarcasm n...|
|             3dRose|3dRose Cool Trend...|Cool Trendy Fashi...|     16.18|     16.18|Arts, Crafts & Se...|     TRUE|[3dRose Sarcasm n...|
|             3dRose|3dRose Sailbo

In [35]:
# 6. Count the number of products which are not available.
win_navailable_df=product_df.withColumn("Product List",F.collect_list(F.col("Product Name")).over(windowSpec))\
    .filter(F.col("Available")=="FALSE")
win_navailable_df.show()

+-------------------+--------------------+--------------------+----------+----------+--------------------+---------+--------------------+
|              Brand|        Product Name|         Description|List Price|Sale Price|            Category|Available|        Product List|
+-------------------+--------------------+--------------------+----------+----------+--------------------+---------+--------------------+
|          100% Pure|100% Pure Long La...|100% Pure Long La...|        30|        30|Beauty|Makeup|Eye...|    FALSE|[100% Pure Long L...|
|          32 Square|32 Square DIL14GW...||Your little pale...|    143.03|    143.03|Toys|Games & Puzz...|    FALSE|[32 Square DIL14G...|
|             3dRose|3dRose Sarcasm no...|Sarcasm noun the ...|      2.49|      2.49|Party & Occasions...|    FALSE|[3dRose Sarcasm n...|
|             3dRose|3dRose Funny Cute...|Funny Cute Hyena ...|      2.49|      2.49|Party & Occasions...|    FALSE|[3dRose Sarcasm n...|
|             3dRose|3dRose candy 

In [ ]:
# 7. List the products that are made up of Nylon.